# Tools

LLMs use various tools to achieve specific goals, streamline operations, and automate tasks. These tools include:

1. **Data retrieval tools:** Extract information from systems or databases using APIs, SDKs, and real-time metrics.
2. **Communication tools:** Facilitate data exchange with external stakeholders via emails, notifications, or alerts.
3. **Data manipulation tools:** Update or modify data within systems, often requiring approval to manage operational impacts.

Additional tools also exist to handle tasks LLMs struggle with, like performing calculations or accessing current date and time.

In [ ]:
from pydantic import BaseModel, ValidationError
from typing import Any, Callable
from language_models.models.llm import OpenAILanguageModel, ChatMessage, ChatMessageRole
from language_models.proxy_client import ProxyClient
from language_models.settings import settings

In [ ]:
proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

In [ ]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-4",
    max_tokens=256,
    temperature=0.2,
)

In [ ]:
class Tool(BaseModel):

    def args(self) -> dict[str, Any] | None:
        pass

    def parse_input(self, tool_input: dict[str, Any]) -> dict[str, Any]:
        pass

    def invoke(self, tool_input: dict[str, Any]) -> Any:
        pass

    def __str__(self) -> str:
        pass

In [ ]:
class Tool(BaseModel):
    args_schema: type[BaseModel] | None = None

    def args(self) -> dict[str, Any] | None:
        if self.args_schema is None:
            return
        return self.args_schema.model_json_schema()["properties"]

In [ ]:
class Tool(BaseModel):
    args_schema: type[BaseModel] | None = None

    def args(self) -> dict[str, Any] | None:
        if self.args_schema is None:
            return
        return self.args_schema.model_json_schema()["properties"]

    def parse_input(self, tool_input: dict[str, Any]) -> dict[str, Any]:
        """Converts tool input to pydantic model."""
        input_args = self.args_schema
        if input_args is not None:
            result = input_args.model_validate(tool_input)
            return {key: getattr(result, key) for key, _ in result.model_dump().items() if key in tool_input}
        return tool_input

In [ ]:
class Tool(BaseModel):
    function: Callable[[Any], Any]
    args_schema: type[BaseModel] | None = None

    def args(self) -> dict[str, Any] | None:
        if self.args_schema is None:
            return
        return self.args_schema.model_json_schema()["properties"]

    def parse_input(self, tool_input: dict[str, Any]) -> dict[str, Any]:
        """Converts tool input to pydantic model."""
        input_args = self.args_schema
        if input_args is not None:
            result = input_args.model_validate(tool_input)
            return {key: getattr(result, key) for key, _ in result.model_dump().items() if key in tool_input}
        return tool_input

    def invoke(self, tool_input: dict[str, Any]) -> Any:
        """Invokes a tool given arguments provided by an LLM."""
        try:
            parsed_input = self._parse_input(tool_input)
            observation = self.function(**parsed_input) if parsed_input else self.function()
        except ValidationError:
            observation = (
                f"Could not run tool {self.name} with input: {tool_input}\n\n"
                + "Your goal is to correct your response\n\n"
                + "Your <input of the tool to use> must be a JSON format with the "
                + f"keyword arguments of: {self.args}"
            )
        return observation

In [ ]:
class Tool(BaseModel):
    function: Callable[[Any], Any]
    name: str
    description: str
    args_schema: type[BaseModel] | None = None

    def args(self) -> dict[str, Any] | None:
        if self.args_schema is None:
            return
        return self.args_schema.model_json_schema()["properties"]

    def parse_input(self, tool_input: dict[str, Any]) -> dict[str, Any]:
        """Converts tool input to pydantic model."""
        input_args = self.args_schema
        if input_args is not None:
            result = input_args.model_validate(tool_input)
            return {key: getattr(result, key) for key, _ in result.model_dump().items() if key in tool_input}
        return tool_input

    def invoke(self, tool_input: dict[str, Any]) -> Any:
        """Invokes a tool given arguments provided by an LLM."""
        try:
            parsed_input = self._parse_input(tool_input)
            observation = self.function(**parsed_input) if parsed_input else self.function()
        except ValidationError:
            observation = (
                f"Could not run tool {self.name} with input: {tool_input}\n\n"
                + "Your goal is to correct your response\n\n"
                + "Your <input of the tool to use> must be a JSON format with the "
                + f"keyword arguments of: {self.args}"
            )
        return observation

    def __str__(self) -> str:
        return f"- Tool Name: {self.name}, " f"Tool Description: {self.description}, " f"Tool Input: {self.args}"